# INFO 2950 Phase 2: Data Collection/Cleanup

In [1]:
import os
import json
import pandas as pd

In [2]:
data_dir = "../data"

if not os.path.exists(data_dir):
    os.mkdir(data_dir)

## Instagram Statistics

We have a directory [`instagram_raw/`](instagram_raw) of `.out` files that we want to convert to dataframes, clean up, and convert to csv files.

In [3]:
instagram_dir = os.path.join(data_dir, "instagram_raw") # instagram dataset
instagram_files = os.listdir(instagram_dir)

instagram_files[:10]

['uchicago.out',
 'yale.out',
 'brownu.out',
 'dartmouthcollege.out',
 'haverfordedu.out',
 'ivyleague.out',
 'ucla.out',
 'bowdoincollege.out',
 'dukeuniversity.out',
 'georgetownuniversity.out']

In [4]:
instagram_df = {}
for file in instagram_files:
    instagram_df[file.split(".")[0]] = pd.read_csv(os.path.join(instagram_dir, file), header=None)

cornell = instagram_df["cornelluniversity"]
cornell

,0,1,2,3
0,2019-09-08 15:42:32.026749,190.5k,160,1773
1,2019-09-09 12:00:03.187746,190.7k,160,1774
2,2019-09-10 12:00:02.818769,190.9k,160,1775
3,2019-09-11 12:00:02.450364,191.1k,160,1777
4,2019-09-12 12:00:03.330241,191.4k,160,1777
...,...,...,...,...
388,2020-10-04 12:00:03.282304,236.2k,176,1934
389,2020-10-05 12:00:02.256002,236.3k,176,1934
390,2020-10-06 12:00:02.470034,236.4k,176,1934
391,2020-10-07 12:00:02.925742,236.6k,176,1934


First, we add an appropriate header to each dataframe. Then, we convert the date strings to date objects.

In [5]:
print(instagram_df["cornelluniversity"].dtypes[0])

for df in instagram_df.values():
    df.columns = ["date", "followers", "following", "posts"]
    df.date = pd.to_datetime(df.date)
    
print(instagram_df["cornelluniversity"].dtypes[0])

cornell.head()

object
datetime64[ns]


,date,followers,following,posts
0,2019-09-08 15:42:32.026749,190.5k,160,1773
1,2019-09-09 12:00:03.187746,190.7k,160,1774
2,2019-09-10 12:00:02.818769,190.9k,160,1775
3,2019-09-11 12:00:02.450364,191.1k,160,1777
4,2019-09-12 12:00:03.330241,191.4k,160,1777


Next, since Instagram gave us non-granular data (eg. 100k instead of 100,000), we apply a function that converts string representations of follower, following, and post counts to integers.

Then, we use `to_numeric` to coerce non-numeric values to `NaN`. This is necessary because some data is corrupt (as a result of rate limiting and bugs in error handling).  
eg. `claremontmckennacollege.out`:
```
...
2020-03-26 12:00:52.845447,5032,462,858
2020-03-27 12:00:53.272308,5035,462,858
2020-03-28 12:00:49.209554,5033,462,858
2020-03-29 12:00:59.842097,5036,463,858
2020-03-30 12:00:53.016625,5041,463,858
2020-03-31 12:00:58.304035,Claremont,College,858
2020-04-01 12:01:02.694789,Claremont,College,858
2020-04-02 12:01:03.518772,Claremont,College,858
2020-04-03 12:01:30.724012,Claremont,College,858
2020-04-04 12:04:34.181228,Claremont,College,860
...
```

In [6]:
# adapted from https://stackoverflow.com/a/41028390/
def to_integer(x):
    try:
        if type(x) == float or type(x) == int:
            return int(x)
        if x[-1] == 'k':
            return int(float(x.replace('k', '')) * 1000)
        if x[-1] == 'm':
            return int(float(x.replace('m', '')) * 1000000)
        return x
    except:
        return x

for df in instagram_df.values():
    df.followers = df.followers.apply(to_integer)
    df.following = df.following.apply(to_integer)
    df.posts = df.posts.apply(to_integer)
    
    df.followers = pd.to_numeric(df.followers, errors='coerce')
    df.following = pd.to_numeric(df.following, errors='coerce')
    df.posts = pd.to_numeric(df.posts, errors='coerce')

print(cornell.dtypes)
cornell.head()

date         datetime64[ns]
followers             int64
following             int64
posts                 int64
dtype: object


,date,followers,following,posts
0,2019-09-08 15:42:32.026749,190500,160,1773
1,2019-09-09 12:00:03.187746,190700,160,1774
2,2019-09-10 12:00:02.818769,190900,160,1775
3,2019-09-11 12:00:02.450364,191100,160,1777
4,2019-09-12 12:00:03.330241,191400,160,1777


Finally, we write this cleaned data to csv files in the directory [`instagram/`](instagram).

In [7]:
instagram_dir_clean = os.path.join(data_dir, "instagram") # clean data output directory

if not os.path.exists(instagram_dir_clean):
    os.mkdir(instagram_dir_clean)

for (name, df) in instagram_df.items():
    df.to_csv(os.path.join(instagram_dir_clean, name + ".csv"), index=False)

pd.read_csv(os.path.join(instagram_dir_clean, "cornelluniversity.csv")).head()

,date,followers,following,posts
0,2019-09-08 15:42:32.026749,190500,160,1773
1,2019-09-09 12:00:03.187746,190700,160,1774
2,2019-09-10 12:00:02.818769,190900,160,1775
3,2019-09-11 12:00:02.450364,191100,160,1777
4,2019-09-12 12:00:03.330241,191400,160,1777


## College/University Information

In [8]:
scorecard = pd.read_json(os.path.join(data_dir, 'scorecard_raw.json'), orient='records')

Rename the columns and verify datatypes

In [9]:
column_rename_map = {
    "id": "id",
    "location.lat": "lat",
    "location.lon": "lon",
    "school.name": "name",
    "school.city": "city",
    "school.state": "state",
    "school.zip": "zip",
    "school.ownership": "ownership",
    "school.region_id": "region",
    "school.locale": "locale",
    "school.carnegie_undergrad": "carnegie_undergrad",
    "school.carnegie_size_setting": "carnegie_size_setting",
    "latest.student.size": "size",
    "latest.student.demographics.avg_family_income": "income_avg",
    "latest.student.demographics.median_family_income": "income_med",
    "latest.cost.avg_net_price.private": "net_price_private",
    "latest.cost.avg_net_price.public": "net_price_public",
    "latest.cost.attendance.academic_year": "cost_attendance",
    "latest.admissions.sat_scores.average.overall": "sat_score",
    "latest.admissions.admission_rate.overall": "admission_rate"
}

scorecard = scorecard.rename(columns=column_rename_map)

scorecard.dtypes

id                         int64
admission_rate           float64
sat_score                float64
cost_attendance          float64
net_price_private        float64
net_price_public         float64
income_avg               float64
income_med               float64
size                     float64
lat                      float64
lon                      float64
carnegie_size_setting      int64
carnegie_undergrad         int64
city                      object
locale                     int64
name                      object
ownership                  int64
region                     int64
state                     object
zip                       object
dtype: object

Map integers representing ownership/region to category strings, and split locale into first and second digit that represent locale type and locale size

In [10]:
ownership_map = {
    1: "public",
    2: "private non-profit",
    3: "private for-profit"
}

region_map = {
    0: "service schools",
    1: "new england",
    2: "mid east",
    3: "great lakes",
    4: "plains",
    5: "southeast",
    6: "southwest",
    7: "rocky mountains",
    8: "far west",
    9: "outlying areas"
}

locale_type_map = {
    1: "city",
    2: "suburb",
    3: "town",
    4: "rural"
}

locale_size_map = {
    1: "large",
    2: "medium",
    3: "small"
}

scorecard['ownership'] = scorecard['ownership'].map(ownership_map)
scorecard['region'] = scorecard['region'].map(region_map)
scorecard['locale_type'] = (scorecard['locale'] // 10).map(locale_type_map)
scorecard['locale_size'] = (scorecard['locale'] % 10).map(locale_size_map)

scorecard.head()

,id,admission_rate,sat_score,cost_attendance,net_price_private,net_price_public,income_avg,income_med,size,lat,...,carnegie_undergrad,city,locale,name,ownership,region,state,zip,locale_type,locale_size
0,147244,0.6126,1125.0,46026.0,20560.0,NaN,66334.0,49741.0,1918.0,39.842612,...,13,Decatur,13,Millikin University,private non-profit,great lakes,IL,62522-2084,city,small
1,147341,0.6937,1114.0,47726.0,17009.0,NaN,70047.0,59988.0,901.0,40.914824,...,12,Monmouth,32,Monmouth College,private non-profit,great lakes,IL,61462-1998,town,medium
2,145691,0.7593,NaN,42221.0,16625.0,NaN,67598.0,59194.0,971.0,39.731300,...,14,Jacksonville,32,Illinois College,private non-profit,great lakes,IL,62650-2299,town,medium
3,148131,0.6815,1079.0,39884.0,13869.0,NaN,64558.0,50859.0,916.0,39.938898,...,13,Quincy,33,Quincy University,private non-profit,great lakes,IL,62301-2699,town,small
4,146667,0.6351,1113.0,25766.0,17727.0,NaN,52255.0,46593.0,398.0,40.148924,...,7,Lincoln,32,Lincoln Christian University,private non-profit,great lakes,IL,62656-2111,town,medium


Simplifying the dataset even further

In [11]:
scorecard = scorecard.drop(columns=['id', 'net_price_private', 'net_price_public', 'carnegie_undergrad', 'carnegie_size_setting', 'locale', 'zip'])
scorecard.head()

,admission_rate,sat_score,cost_attendance,income_avg,income_med,size,lat,lon,city,name,ownership,region,state,locale_type,locale_size
0,0.6126,1125.0,46026.0,66334.0,49741.0,1918.0,39.842612,-88.976298,Decatur,Millikin University,private non-profit,great lakes,IL,city,small
1,0.6937,1114.0,47726.0,70047.0,59988.0,901.0,40.914824,-90.637310,Monmouth,Monmouth College,private non-profit,great lakes,IL,town,medium
2,0.7593,NaN,42221.0,67598.0,59194.0,971.0,39.731300,-90.246171,Jacksonville,Illinois College,private non-profit,great lakes,IL,town,medium
3,0.6815,1079.0,39884.0,64558.0,50859.0,916.0,39.938898,-91.385975,Quincy,Quincy University,private non-profit,great lakes,IL,town,small
4,0.6351,1113.0,25766.0,52255.0,46593.0,398.0,40.148924,-89.346721,Lincoln,Lincoln Christian University,private non-profit,great lakes,IL,town,medium


In [12]:
scorecard.to_csv(os.path.join(data_dir, "scorecard.csv"), index=False)

## Merged Dataset

We want to do a left join on the Instagram and Scorecard datasets. However, our Instagram dataset consists of multiple dataframes. To merge it into a 2d dataframe, we're ommitting the time variable and opting to look at summary statistics.

In [13]:
college_names = []
current_follower_counts = []

for (name, df) in instagram_df.items():
    followers = df.followers
    
    current_follower_count = followers.iloc[-1]
    
    college_names.append(name)
    current_follower_counts.append(current_follower_count)

instagram_summary = pd.DataFrame(data={"instagram": college_names, "follower_count": current_follower_counts})
instagram_summary = instagram_summary.sort_values(by=["follower_count"], ascending=False).reset_index(drop=True)
instagram_summary.head()

,instagram,follower_count
0,harvard,1700000.0
1,stanford,799300.0
2,yale,490700.0
3,princeton,352700.0
4,mitpics,308300.0


In [14]:
with open('username_map.json') as username_map_json:
    username_map = json.load(username_map_json)

instagram_summary["name"] = instagram_summary["instagram"].map(username_map)
instagram_summary.head()

,instagram,follower_count,name
0,harvard,1700000.0,Harvard University
1,stanford,799300.0,Stanford University
2,yale,490700.0,Yale University
3,princeton,352700.0,Princeton University
4,mitpics,308300.0,Massachusetts Institute of Technology


In [15]:
detailed_instagram = instagram_summary.merge(scorecard, how="left", on="name")
detailed_instagram

,instagram,follower_count,name,admission_rate,sat_score,cost_attendance,income_avg,income_med,size,lat,lon,city,ownership,region,state,locale_type,locale_size
0,harvard,1700000.0,Harvard University,0.0473,1520.0,71135.0,62458.0,33066.0,7582.0,42.374471,-71.118313,Cambridge,private non-profit,new england,MA,city,medium
1,stanford,799300.0,Stanford University,0.0436,1497.0,69109.0,80447.0,44842.0,7083.0,37.429434,-122.167359,Stanford,private non-profit,far west,CA,suburb,large
2,yale,490700.0,Yale University,0.0635,1517.0,71290.0,80258.0,44004.0,5963.0,41.311158,-72.926688,New Haven,private non-profit,new england,CT,city,medium
3,princeton,352700.0,Princeton University,0.0548,1503.0,66950.0,63667.0,37036.0,5301.0,40.348732,-74.659365,Princeton,private non-profit,mid east,NJ,suburb,large
4,mitpics,308300.0,Massachusetts Institute of Technology,0.0674,1545.0,67430.0,86738.0,53870.0,4550.0,42.359243,-71.093226,Cambridge,private non-profit,new england,MA,city,medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,grinnellcollege,6516.0,Grinnell College,0.2438,1450.0,65814.0,90298.0,74915.0,1683.0,41.747952,-92.722094,Grinnell,private non-profit,plains,IA,town,small
66,claremontmckennacollege,6095.0,Claremont McKenna College,0.0931,1446.0,71745.0,97309.0,71150.0,1318.0,34.102154,-117.711188,Claremont,private non-profit,far west,CA,suburb,large
67,pitzercollege,4738.0,Pitzer College,0.1333,NaN,70500.0,93983.0,71480.0,1072.0,34.104107,-117.706675,Claremont,private non-profit,far west,CA,suburb,large
68,harvey_mudd,3228.0,Harvey Mudd College,0.1448,1531.0,74428.0,123783.0,104805.0,886.0,34.106515,-117.709837,Claremont,private non-profit,far west,CA,suburb,large
